In [1]:
!pip install opencv-python

In [2]:
!pip install tqdm

In [3]:
import os
import pathlib
import cv2
from tqdm.notebook import tqdm
import json
import shutil
import math

In [4]:
#동영상 1개당 240프레임씩 추출
MAX_FRAME_NUM = 240

In [5]:
#동영상에서 추출한 프레임을 저장할 폴더
frame_path = f"data/frame/"

#pathlib.Path(frame_path): 프레임을 저장할 폴더의 정보를 알아낼 객체 리턴
frame_save_path = pathlib.Path(frame_path)

#frame_save_path.exists(): 프레임을 저장할 폴더 data/frame이 존재하면
if frame_save_path.exists():
    #프레임을 저장할 폴더 data/frame 삭제
    shutil.rmtree(frame_path)

#학습 동영상 파일들의 경로를 저장할 리스트
mp4_path_list = []

mp4_root_path = "C:/Users/User/Downloads/114.재난 안전 정보 전달을 위한 수어영상 데이터/01.데이터/1.Training/원천데이터"

#mp4_root_path 폴더에 포함된 파일들 리턴
for(path, dir, files) in os.walk(mp4_root_path):
    print("path=", path)
    print("dir=", dir)
    print("files=", files)
    print("=" * 100)

    #path에 포함된 파일 리스트에서 파일명을 filename에 대입
    for filename in files:
        print("filename=", filename)
        #filename을 파일명과 확장자로 분리
        spname = os.path.splitext(filename)
        print("spname=", spname)
        #파일명의 확장자와 파일명 리턴
        ext = spname[-1]
        filename_not_ext = os.path.splitext(filename)[0]
        print("ext=", ext)
        print("filename_not_ext=", filename_not_ext)
        print("=" * 100)

        #확장자가 .mp4일 때
        if ext == '.mp4':
            #파일명의 마지막 글자가 L이 아닐 때(좌측에서 촬영한 수어 영상이 아닐 때)
            #좌측 촬영 수어 영상을 제외한 정면 수어 영상만 학습
            if (filename_not_ext[-1] != 'L') & (filename_not_ext[-1] != 'U') & (filename_not_ext[-1] != 'R') & (filename_not_ext[-1] != 'D' ) :
                file_path = path + "\\" + filename
                print("file_path=", file_path)
                #mp4_path_list에 확장자가 mp4인 파일의 경로 추가
                mp4_path_list.append(file_path)
        print("=" *100)

path= C:/Users/User/Downloads/114.재난 안전 정보 전달을 위한 수어영상 데이터/01.데이터/1.Training/원천데이터
dir= ['1.mp4']
files= []
path= C:/Users/User/Downloads/114.재난 안전 정보 전달을 위한 수어영상 데이터/01.데이터/1.Training/원천데이터\1.mp4
dir= ['1.5angle', '3.crowd']
files= []
path= C:/Users/User/Downloads/114.재난 안전 정보 전달을 위한 수어영상 데이터/01.데이터/1.Training/원천데이터\1.mp4\1.5angle
dir= ['4.기타재난']
files= []
path= C:/Users/User/Downloads/114.재난 안전 정보 전달을 위한 수어영상 데이터/01.데이터/1.Training/원천데이터\1.mp4\1.5angle\4.기타재난
dir= ['MOUNTAINSAFETYACCIDENT', 'SUMMERTIMEWATERGAME']
files= []
path= C:/Users/User/Downloads/114.재난 안전 정보 전달을 위한 수어영상 데이터/01.데이터/1.Training/원천데이터\1.mp4\1.5angle\4.기타재난\MOUNTAINSAFETYACCIDENT
dir= ['1_1']
files= []
path= C:/Users/User/Downloads/114.재난 안전 정보 전달을 위한 수어영상 데이터/01.데이터/1.Training/원천데이터\1.mp4\1.5angle\4.기타재난\MOUNTAINSAFETYACCIDENT\1_1
dir= []
files= ['NIA_SL_G4_MOUNTAINSAFETYACCIDENT000001_1_TW10_D.mp4', 'NIA_SL_G4_MOUNTAINSAFETYACCIDENT000001_1_TW10_F.mp4', 'NIA_SL_G4_MOUNTAINSAFETYACCIDENT000001_1_TW10_L.mp4', 'NIA_S

In [6]:
mp4_path_list

['C:/Users/User/Downloads/114.재난 안전 정보 전달을 위한 수어영상 데이터/01.데이터/1.Training/원천데이터\\1.mp4\\1.5angle\\4.기타재난\\MOUNTAINSAFETYACCIDENT\\1_1\\NIA_SL_G4_MOUNTAINSAFETYACCIDENT000001_1_TW10_F.mp4',
 'C:/Users/User/Downloads/114.재난 안전 정보 전달을 위한 수어영상 데이터/01.데이터/1.Training/원천데이터\\1.mp4\\1.5angle\\4.기타재난\\MOUNTAINSAFETYACCIDENT\\1_1\\NIA_SL_G4_MOUNTAINSAFETYACCIDENT000002_1_TW10_F.mp4',
 'C:/Users/User/Downloads/114.재난 안전 정보 전달을 위한 수어영상 데이터/01.데이터/1.Training/원천데이터\\1.mp4\\1.5angle\\4.기타재난\\MOUNTAINSAFETYACCIDENT\\1_1\\NIA_SL_G4_MOUNTAINSAFETYACCIDENT000003_1_TW10_F.mp4',
 'C:/Users/User/Downloads/114.재난 안전 정보 전달을 위한 수어영상 데이터/01.데이터/1.Training/원천데이터\\1.mp4\\1.5angle\\4.기타재난\\MOUNTAINSAFETYACCIDENT\\1_1\\NIA_SL_G4_MOUNTAINSAFETYACCIDENT000004_1_TW10_F.mp4',
 'C:/Users/User/Downloads/114.재난 안전 정보 전달을 위한 수어영상 데이터/01.데이터/1.Training/원천데이터\\1.mp4\\1.5angle\\4.기타재난\\MOUNTAINSAFETYACCIDENT\\1_1\\NIA_SL_G4_MOUNTAINSAFETYACCIDENT000005_1_TW10_F.mp4',
 'C:/Users/User/Downloads/114.재난 안전 정보 전달을 위한 수어영상 데이터/01.데이

In [7]:
for mp4_path in mp4_path_list[0:2]:
    #mp4_path_list에 저장된 mp4 파일 경로를 분리
    sp1 = mp4_path.split("\\")
    
    #파일명과 확장자를 제외한 문자를 리턴
    video_name = sp1[-1]
    video_name_not_ext = video_name[:-4]

    #mp4 파일의 정보를 가져올 객체 생성
    capture = cv2.VideoCapture(mp4_path)

    #mp4 파일의 프레임 개수 조회
    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    print("frame_count=", frame_count)

    #동영상 1개당 240프레임씩 저장하기 위해
    #전체 동영상 개수 frame_count를 MAX_FRAME_NUM으로 나눔
    skip_num = frame_count / MAX_FRAME_NUM
    #나눈 값을 올림처리
    skip_num = math.ceil(skip_num)

    #프레임을 저장할 폴더 경로
    frame_path = f"data/frame/{video_name_not_ext}"

    #프레임을 저장할 폴더의 정보를 알아낼 객체 생성
    frame_save_path = pathlib.Path(frame_path)

    #프레임을 저장할 폴더가 존재하지 않으면 폴더 생성
    if not frame_save_path.exists():
        os.makedirs(frame_save_path)

    #동영상의 프레임 번호
    frame_num = 0
    #저장된 jpg 순서
    jpg_num = 0

    while(True):
        #동영상의 프레임을 읽어서 저장
        success, frame = capture.read()

        if success & (frame_num % skip_num == 0) & (jpg_num < MAX_FRAME_NUM):
            jpg_path = f'{frame_save_path}/{video_name_not_ext}_{jpg_num}.jpg'
            cv2.imwrite(jpg_path, frame)
            jpg_num = jpg_num + 1

        if not success:
            break

        frame_num = frame_num + 1

    #비디오 파일 닫기
    capture.release()
    print("=" * 100)

frame_count= 501
frame_count= 541


In [8]:
for mp4_path in tqdm(mp4_path_list):

    try:
        sp1 = mp4_path.split("\\")
        video_name = sp1[-1]
        video_name_not_ext = video_name[:-4]

        capture = cv2.VideoCapture(mp4_path)
        frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
        skip_num = frame_count / MAX_FRAME_NUM
        skip_num = math.ceil(skip_num)

        frame_path = f"data/frame/{video_name_not_ext}"
        frame_save_path = pathlib.Path(frame_path)

        if not frame_save_path.exists():
            os.makedirs(frame_save_path)
            
        frame_num = 0
        jpg_num = 0
        
        while(True):
            success, frame = capture.read()
            
            if success & (frame_num % skip_num == 0) & (jpg_num < MAX_FRAME_NUM):
                jpg_path = f'{frame_save_path}/{video_name_not_ext}_{jpg_num}.jpg'
                cv2.imwrite(jpg_path, frame)
                jpg_num = jpg_num + 1

            if not success:
                break
                
            frame_num = frame_num + 1

        capture.release()
    
    except Exception as e:
        print("=" * 100)
        print('예외 발생', e)
        print("=" * 100)

  0%|          | 0/352 [00:00<?, ?it/s]